In [1]:
# ==========================================
# 0. INSTALL DEPENDENCIES (Run this once)
# ==========================================
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets sentence-transformers faiss-cpu tqdm

import os
import json
import pandas as pd
from unsloth import FastLanguageModel
from google.colab import drive
import torch
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8dd8c3kk/unsloth_5c993727dd0144c5a876d7a4917ec5eb
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8dd8c3kk/unsloth_5c993727dd0144c5a876d7a4917ec5eb
  Resolved https://github.com/unslothai/unsloth.git to commit ec1757c1a02175851146ff5f6ab2a26c8c863fc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Hugging Face Login (Required for gated models like Gemma/Llama)
from huggingface_hub import login
# Replace 'YOUR_HF_TOKEN' with your actual token or use Colab secrets
# login(token="YOUR_HF_TOKEN")

# Paths
# Use the official ID for downloading directly in Colab
# Using Llama-3-8B-Instruct (Unsloth version is safer for Colab free tier than Meta-Llama-3.1-8B)
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
input_path = '/content/task-a-en.tsv'
output_file = "/content/outputs_llama_rag_new1.jsonl"

import numpy as np

In [3]:
# ==========================================
# 2. LOAD RAG SYSTEM (Retriever)
# ==========================================
class HFRetriever:
    def __init__(self, n_docs=20000):
        print(f"Loading embedded Wikipedia subset: {n_docs} docs ...")
        # Load pre-embedded dataset (fast)
        self.ds = load_dataset(
            "not-lain/wikipedia",
            revision="embedded",
            split=f"train[:{n_docs}]"
        )
        self.texts = [str(x) for x in self.ds["text"]]
        # Load embeddings into numpy (fast cosine sim)
        self.embs = np.array(self.ds["embeddings"], dtype=np.float32)

        # Normalize doc vectors once
        self.embs = self.embs / (np.linalg.norm(self.embs, axis=1, keepdims=True) + 1e-12)

        print("Loading query embedding model...")
        # Use the exact same model that was used to embed the dataset!
        # The dataset 'not-lain/wikipedia' was embedded with 'mixedbread-ai/mxbai-embed-large-v1'
        # This model outputs 1024-dim vectors.
        # We CANNOT use 'all-MiniLM-L6-v2' (384-dim) because dimensions mismatch.
        self.encoder = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
        print("RAG Index Ready!")

    def retrieve(self, query, k=3, max_chars=1000):
        try:
            # Encode query
            q = self.encoder.encode([query]).astype(np.float32)
            q = q / (np.linalg.norm(q, axis=1, keepdims=True) + 1e-12)

            # Fast Cosine Similarity
            sims = self.embs @ q[0]
            top_idx = np.argsort(-sims)[:k]

            # Combine results
            combined_text = "\n\n".join([self.texts[i] for i in top_idx])
            return combined_text[:max_chars]
        except Exception as e:
            print(f"Retrieval failed: {e}")
            return ""

# Initialize Retriever (Fast Mode)
retriever = HFRetriever(n_docs=20000)


Loading embedded Wikipedia subset: 20000 docs ...
Loading query embedding model...
RAG Index Ready!


In [5]:
# ==========================================
# 3. LOAD LLAMA MODEL
# ==========================================
print(f"Loading Llama from {model_id}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype = None,
)

FastLanguageModel.for_inference(model)

Loading Llama from unsloth/llama-3-8b-Instruct-bnb-4bit...
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm):

In [9]:
# ==========================================
# 4. GENERATION LOOP
# ==========================================
# Read Input
df = pd.read_csv(input_path, sep='\t')
data = df.to_dict('records')

# Resume Check
processed_ids = set()
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                saved_item = json.loads(line)
                processed_ids.add(saved_item['id'])
            except: pass
    print(f"Resuming... Found {len(processed_ids)} jokes.")

print("Starting RAG Generation with Llama...")

for row in tqdm(data, desc="Generating Jokes"):
    current_id = row['id']
    if current_id in processed_ids: continue

    # Parse Input
    headline_val = str(row.get('headline', "-")).strip()
    w1_val = str(row.get('word1', "-")).strip()
    w2_val = str(row.get('word2', "-")).strip()

    # --- RAG RETRIEVAL ---
    if headline_val != "-" and headline_val != "" and headline_val.lower() != "nan":
        # Search for the headline content
        context = retriever.retrieve(headline_val)

        # Prompt with Context
        prompt_text = f"""### Instruction
You are a witty, cynical stand-up comedian.
Use the following background information ONLY if it helps inspire a joke. Otherwise, ignore it.

Background Info:
{context}

Headline: "{headline_val}"

Task: Write EXACTLY ONE punchy joke (1-2 sentences) about this headline.
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.

### Response
Joke:"""
        input_type = "headline"
        input_content = headline_val

    else:
        # Search for the words
        real_w1 = w1_val if w1_val != "-" else "something"
        real_w2 = w2_val if w2_val != "-" else "random"
        query = f"{real_w1} {real_w2}"
        context = retriever.retrieve(query)

        prompt_text = f"""### Instruction
You are a witty, cynical stand-up comedian.
Use the following background information ONLY if it helps inspire a joke. Otherwise, ignore it.

Background Info:
{context}

Task: Write EXACTLY ONE punchy joke (1-2 sentences) connecting: "{real_w1}" and "{real_w2}".
- Be clever, cynical, or ironic.
- Do NOT explain the joke.
- Output ONLY the joke.

### Response
Joke:"""
        input_type = "words"
        input_content = f"{real_w1}, {real_w2}"

    # --- GENERATE ---
    # Robust: Use direct tokenization, not apply_chat_template which might fail
    inputs = tokenizer([prompt_text], return_tensors="pt").to("cuda")

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        do_sample = True,
        max_new_tokens = 128,
        temperature = 0.9,
        top_p = 0.9,
        repetition_penalty = 1.2
    )

    # --- PARSE ---
    decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 1. Remove the prompt (input) from the output so we don't parse it by mistake
    if "### Response" in decoded_text:
        response_part = decoded_text.split("### Response")[-1]
    else:
        response_part = decoded_text

    # 2. Find "Joke:"
    if "Joke:" in response_part:
        final_joke = response_part.split("Joke:")[-1].strip()
    else:
        # Fallback: Just take the whole response part
        final_joke = response_part.strip()

    # 3. Cleanup: Take first paragraph only (stop at double newline)
    final_joke = final_joke.split("\n\n")[0].strip().strip('"').strip("'")

    # 4. Remove artifacts like "assistant"
    if final_joke.lower().startswith("assistant"):
        final_joke = final_joke[9:].strip().lstrip(":").strip()

    # If joke is still empty or just "assistant", fallback to raw text
    if not final_joke or final_joke.lower() == "assistant":
        final_joke = decoded_text.replace(prompt_text, "").strip().split("\n\n")[0]

    # Preview
    print(f"ID {current_id} [RAG]: {final_joke[:50]}...")

    # Save
    result_entry = {
        "id": current_id,
        "type": input_type,
        "input_original": input_content,
        "retrieved_context": context[:600] + "...",
        "generated_joke": final_joke
    }

    with open(output_file, "a", encoding='utf-8') as f:
        f.write(json.dumps(result_entry, ensure_ascii=False) + "\n")

print("Finished RAG Generation!")


Starting RAG Generation with Llama...


Generating Jokes:   0%|          | 1/1200 [00:02<44:57,  2.25s/it]

ID en_0001 [RAG]: Ryanair's new slogan must be "Fly low-cost, cry yo...


Generating Jokes:   0%|          | 2/1200 [00:04<45:39,  2.29s/it]

ID en_0002 [RAG]: I guess 'looted' means what they used to do at auc...


...

ID en_1198 [RAG]: So now we know what they mean when they say 'flyin...


Generating Jokes: 100%|█████████▉| 1199/1200 [49:27<00:02,  2.31s/it]

ID en_1199 [RAG]: Lakefront entertainment – because 'I'm stuck watch...


Generating Jokes: 100%|██████████| 1200/1200 [49:30<00:00,  2.48s/it]

ID en_1200 [RAG]: So now we know why London's got more foreign accen...
Finished RAG Generation!
